In [105]:
def extract_all_company_data(file_path):
    csv_files = []  # List to store .csv file names

    # Get a list of all files and directories in the specified directory
    for file in os.listdir(file_path):
        if file.endswith(".csv"):
            csv_path = os.path.join(file_path, file)
            df = pd.read_csv(csv_path)
            print(f'{file}: {len(df)} rows')

    company_data = []

    # Iterate through the CSV files
    for csv_file in csv_files:
        # Read the CSV file into a DataFrame
        csv_path = os.path.join(file_path, csv_file)
        df = pd.read_csv(csv_path)

        # Append each row to the company_data list
        for index, row in df.iterrows():
            company_data.append(row.to_dict())  # Convert each row to a dictionary and append

    return pd.DataFrame(company_data)

# Call the modified function to print row counts for each CSV
extract_all_company_data(file_path)

audi.csv: 10668 rows
bmw.csv: 10781 rows
cclass.csv: 3899 rows
focus.csv: 5454 rows
ford.csv: 17965 rows
hyundi.csv: 4860 rows
merc.csv: 13119 rows
skoda.csv: 6267 rows
toyota.csv: 6738 rows
unclean cclass.csv: 4006 rows
unclean focus.csv: 5604 rows
vauxhall.csv: 13632 rows
vw.csv: 15157 rows


""


In [27]:
import os
import pandas as pd

def extract_all_company_data(file_path_read, file_path_write, exclude_files):
    csv_files = []  # List to store .csv file names
    
    # Define a dictionary to map brands to countries
    brand_country_mapping = {
        'Audi': 'Germany',
        'BMW': 'Germany',
        'Ford': 'United States',
        'Hyundai': 'South Korea',
        'Mercedes-Benz': 'Germany',
        'Skoda': 'Czech Republic',
        'Toyota': 'Japan',
        'Vauxhall': 'United Kingdom',
        'Volkswagen': 'Germany'
    }
      
    # Get a list of all files and directories in the specified directory
    for file in os.listdir(file_path_read):
        if file.endswith(".csv") and file not in exclude_files:
            csv_files.append(file)  # Add .csv file to the list
    
    company_data = []
    
    # Iterate through the CSV files
    for csv_file in (csv_files[:2]):
        # Read the CSV file into a DataFrame
        csv_path = os.path.join(file_path_read, csv_file)
        df = pd.read_csv(csv_path)
        
        # Get the manufacturer name (assuming the CSV filename is the manufacturer name)
        brand_name = os.path.splitext(os.path.basename(csv_file))[0]
        if brand_name.lower() == 'mercedes-benz':
            brand_name = 'Mercedes-Benz'
        elif brand_name.lower() == 'bmw':
            brand_name = 'BMW'
        else:
            brand_name = brand_name.capitalize()
                  
        # Add a column for the country based on brand as the first column
        df.insert(0, 'country', brand_country_mapping.get(brand_name, None))
        
        # Add a column for the brand name
        df.insert(1, 'brand', brand_name)

        # Save the modified DataFrame to a new CSV file in the 'selected' folder
        output_file_path = os.path.join(file_path_write, f"{brand_name.lower()}_modified.csv")
        df.to_csv(output_file_path, index=False)
        
        # Append each row to the company_data list
        for index, row in df.iterrows():
            company_data.append(row.to_dict())  # Convert each row to a dictionary and append
        
    return pd.DataFrame(company_data)

# Define the path to read from
file_path_read = 'data/raw_renamed'
# Define the path to write to (inside the 'selected' folder)
file_path_write = 'data/selected'
# Define the files to be excluded
exclude_files = ['cclass.csv', 'focus.csv', 'unclean cclass.csv', 'unclean focus.csv']

# Call the function to process and save each individual CSV
extract_all_company_data(file_path_read, file_path_write, exclude_files)

,country,brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Germany,Audi,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,Germany,Audi,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,Germany,Audi,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,Germany,Audi,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,Germany,Audi,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0
...,...,...,...,...,...,...,...,...,...,...,...
21444,Germany,BMW,X3,2016,19000,Automatic,40818,Diesel,150,54.3,2.0
21445,Germany,BMW,5 Series,2016,14600,Automatic,42947,Diesel,125,60.1,2.0
21446,Germany,BMW,3 Series,2017,13100,Manual,25468,Petrol,200,42.8,2.0
21447,Germany,BMW,1 Series,2014,9930,Automatic,45000,Diesel,30,64.2,2.0


In [110]:
import pandas as pd

# Load the 'audi.csv' file into a DataFrame
audi_df = pd.read_csv('data/selected/bmw_modified.csv')  # Replace with the actual path to your 'audi.csv' file

# Check the number of rows in the 'audi.csv' DataFrame
num_rows = audi_df.shape[0]  # The number of rows is the first element of the shape tuple

print('Number of rows in audi.csv:', num_rows)


Number of rows in audi.csv: 10781


In [125]:
import requests

def fetch_country_car_mapping_from_api():
    # Wikipedia API endpoint for parsing content
    api_endpoint = "https://en.wikipedia.org/w/api.php"

    # Parameters for the API request to extract content
    params = {
        'action': 'parse',
        'format': 'json',
        'page': 'List_of_current_automobile_manufacturers_by_country',
        'section': 1  # Section number where the content is present
    }

    try:
        # Send a request to the Wikipedia API
        response = requests.get(api_endpoint, params=params)
        response_data = response.json()

        # Check if the response is valid
        if 'parse' not in response_data:
            print("Failed to retrieve data from Wikipedia.")
            return None

        # Extract the content from the parsed data
        content = response_data['parse']['text']['*']

        # Use BeautifulSoup to parse the content
        soup = BeautifulSoup(content, 'html.parser')

        # Process the data and extract country names and corresponding car manufacturers
        country_car_mapping = {}
        current_country = None

        for element in soup.find_all(['h3', 'ul']):
            if element.name == 'h3':
                current_country = element.text.strip()
                country_car_mapping[current_country] = []
            elif element.name == 'ul' and current_country:
                car_manufacturers = element.find_all('li')
                for manufacturer in car_manufacturers:
                    country_car_mapping[current_country].append(manufacturer.text.strip())

        return country_car_mapping

    except requests.exceptions.RequestException as e:
        print("Error: ", e)
        return None

# Fetch country-car mapping using the Wikipedia API
country_car_mapping_api = fetch_country_car_mapping_from_api()

# Print the obtained country-car mapping
for country, manufacturers in country_car_mapping_api.items():
    print(f"Country: {country}")
    print(f"Car Manufacturers: {manufacturers}\n")


Country: Algeria[edit]
Car Manufacturers: ['SNVI']

Country: Argentina[edit]
Car Manufacturers: ['Zanella', 'Koller']

Country: Australia[edit]
Car Manufacturers: ['ACE EV Group', 'Borland Racing', 'Bolwell', 'Brabham Automotive', 'Elfin Sports Cars', 'Ford Australia', 'GMSV']

Country: Austria[edit]
Car Manufacturers: ['Magna Steyr', 'Puch', 'Tushek & Spigel']

Country: Azerbaijan[edit]
Car Manufacturers: ['AzSamand', 'Ganja Auto Plant', 'Nakhchivan Automobile Plant']



In [126]:
country_car_mapping_api

{'Algeria[edit]': ['SNVI'],
 'Argentina[edit]': ['Zanella', 'Koller'],
 'Australia[edit]': ['ACE EV Group',
  'Borland Racing',
  'Bolwell',
  'Brabham Automotive',
  'Elfin Sports Cars',
  'Ford Australia',
  'GMSV'],
 'Austria[edit]': ['Magna Steyr', 'Puch', 'Tushek & Spigel'],
 'Azerbaijan[edit]': ['AzSamand',
  'Ganja Auto Plant',
  'Nakhchivan Automobile Plant']}

In [127]:
import requests
from bs4 import BeautifulSoup

def fetch_country_car_mapping_from_api():
    # Wikipedia API endpoint for parsing content
    api_endpoint = "https://en.wikipedia.org/w/api.php"

    # Dictionary to store country-car mapping
    country_car_mapping = {}

    # Iterate through sections A to Z
    for section_number in range(1, 27):  # 26 sections for each letter of the alphabet
        # Parameters for the API request to extract content for the specific section
        params = {
            'action': 'parse',
            'format': 'json',
            'page': 'List_of_current_automobile_manufacturers_by_country',
            'section': section_number
        }

        try:
            # Send a request to the Wikipedia API
            response = requests.get(api_endpoint, params=params)
            response_data = response.json()

            # Check if the response is valid
            if 'parse' not in response_data:
                print(f"Failed to retrieve data for section {chr(64 + section_number)}")
                continue

            # Extract the content from the parsed data
            content = response_data['parse']['text']['*']

            # Use BeautifulSoup to parse the content
            soup = BeautifulSoup(content, 'html.parser')

            # Process the data and extract country names and corresponding car manufacturers
            current_country = None
            for element in soup.find_all(['h3', 'ul']):
                if element.name == 'h3':
                    current_country = element.text.strip()
                    country_car_mapping.setdefault(current_country, [])
                elif element.name == 'ul' and current_country:
                    car_manufacturers = element.find_all('li')
                    for manufacturer in car_manufacturers:
                        country_car_mapping[current_country].append(manufacturer.text.strip())

        except requests.exceptions.RequestException as e:
            print(f"Error for section {chr(64 + section_number)}: {e}")

    return country_car_mapping

# Fetch country-car mapping using the Wikipedia API
country_car_mapping_api = fetch_country_car_mapping_from_api()

# Print the obtained country-car mapping
for country, manufacturers in country_car_mapping_api.items():
    print(f"Country: {country}")
    print(f"Car Manufacturers: {manufacturers}\n")


Country: Algeria[edit]
Car Manufacturers: ['SNVI', 'SNVI']

Country: Argentina[edit]
Car Manufacturers: ['Zanella', 'Koller', 'Zanella', 'Koller']

Country: Australia[edit]
Car Manufacturers: ['ACE EV Group', 'Borland Racing', 'Bolwell', 'Brabham Automotive', 'Elfin Sports Cars', 'Ford Australia', 'GMSV', 'ACE EV Group', 'Borland Racing', 'Bolwell', 'Brabham Automotive', 'Elfin Sports Cars', 'Ford Australia', 'GMSV']

Country: Austria[edit]
Car Manufacturers: ['Magna Steyr', 'Puch', 'Tushek & Spigel', 'Magna Steyr', 'Puch', 'Tushek & Spigel']

Country: Azerbaijan[edit]
Car Manufacturers: ['AzSamand', 'Ganja Auto Plant', 'Nakhchivan Automobile Plant', 'AzSamand', 'Ganja Auto Plant', 'Nakhchivan Automobile Plant']

Country: Bangladesh[edit]
Car Manufacturers: ['Aftab Automobiles', 'PHP Automobiles', 'Aftab Automobiles', 'PHP Automobiles']

Country: Belarus[edit]
Car Manufacturers: ['BelAZ', 'BelGee', 'MAZ', 'MoAZ', 'MZKT', 'Neman', 'BelAZ', 'BelGee', 'MAZ', 'MoAZ', 'MZKT', 'Neman']

Coun

In [128]:
country_car_mapping_api

{'Algeria[edit]': ['SNVI', 'SNVI'],
 'Argentina[edit]': ['Zanella', 'Koller', 'Zanella', 'Koller'],
 'Australia[edit]': ['ACE EV Group',
  'Borland Racing',
  'Bolwell',
  'Brabham Automotive',
  'Elfin Sports Cars',
  'Ford Australia',
  'GMSV',
  'ACE EV Group',
  'Borland Racing',
  'Bolwell',
  'Brabham Automotive',
  'Elfin Sports Cars',
  'Ford Australia',
  'GMSV'],
 'Austria[edit]': ['Magna Steyr',
  'Puch',
  'Tushek & Spigel',
  'Magna Steyr',
  'Puch',
  'Tushek & Spigel'],
 'Azerbaijan[edit]': ['AzSamand',
  'Ganja Auto Plant',
  'Nakhchivan Automobile Plant',
  'AzSamand',
  'Ganja Auto Plant',
  'Nakhchivan Automobile Plant'],
 'Bangladesh[edit]': ['Aftab Automobiles',
  'PHP Automobiles',
  'Aftab Automobiles',
  'PHP Automobiles'],
 'Belarus[edit]': ['BelAZ',
  'BelGee',
  'MAZ',
  'MoAZ',
  'MZKT',
  'Neman',
  'BelAZ',
  'BelGee',
  'MAZ',
  'MoAZ',
  'MZKT',
  'Neman'],
 'Belgium[edit]': ['Gillet',
  'Van Hool',
  'Edran',
  'Gillet',
  'Van Hool',
  'Edran'],
 'Braz

In [2]:
import requests
from bs4 import BeautifulSoup
import time

def fetch_country_car_mapping_from_api():
    # Wikipedia API endpoint for parsing content
    api_endpoint = "https://en.wikipedia.org/w/api.php"

    # Dictionary to store country-car mapping
    country_car_mapping = {}

    # Iterate through sections A to Z
    for section_number in range(1, 27):  # 26 sections for each letter of the alphabet
        # Parameters for the API request to extract content for the specific section
        params = {
            'action': 'parse',
            'format': 'json',
            'page': 'List_of_current_automobile_manufacturers_by_country',
            'section': section_number
        }

        try:
            # Send a request to the Wikipedia API
            response = requests.get(api_endpoint, params=params)
            response_data = response.json()

            # Check if the response is valid
            if 'parse' not in response_data:
                print(f"Failed to retrieve data for section {chr(64 + section_number)}")
                continue

            # Extract the content from the parsed data
            content = response_data['parse']['text']['*']

            # Use BeautifulSoup to parse the content
            soup = BeautifulSoup(content, 'html.parser')

            # Process the data and extract country names and corresponding car manufacturers
            current_country = None
            for element in soup.find_all(['h3', 'ul']):
                if element.name == 'h3':
                    current_country = element.text.strip()
                    country_car_mapping.setdefault(current_country, [])
                elif element.name == 'ul' and current_country:
                    car_manufacturers = element.find_all('li')
                    for manufacturer in car_manufacturers:
                        country_car_mapping[current_country].append(manufacturer.text.strip())

            # Introduce a delay to stay within API rate limits
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            print(f"Error for section {chr(64 + section_number)}: {e}")

    return country_car_mapping

# Fetch country-car mapping using the Wikipedia API
country_car_mapping_api = fetch_country_car_mapping_from_api()

# Print the obtained country-car mapping
for country, manufacturers in country_car_mapping_api.items():
    print(f"Country: {country}")
    print(f"Car Manufacturers: {manufacturers}\n")


Country: Algeria[edit]
Car Manufacturers: ['SNVI', 'SNVI']

Country: Argentina[edit]
Car Manufacturers: ['Zanella', 'Koller', 'Zanella', 'Koller']

Country: Australia[edit]
Car Manufacturers: ['ACE EV Group', 'Borland Racing', 'Bolwell', 'Brabham Automotive', 'Elfin Sports Cars', 'Ford Australia', 'GMSV', 'ACE EV Group', 'Borland Racing', 'Bolwell', 'Brabham Automotive', 'Elfin Sports Cars', 'Ford Australia', 'GMSV']

Country: Austria[edit]
Car Manufacturers: ['Magna Steyr', 'Puch', 'Tushek & Spigel', 'Magna Steyr', 'Puch', 'Tushek & Spigel']

Country: Azerbaijan[edit]
Car Manufacturers: ['AzSamand', 'Ganja Auto Plant', 'Nakhchivan Automobile Plant', 'AzSamand', 'Ganja Auto Plant', 'Nakhchivan Automobile Plant']

Country: Bangladesh[edit]
Car Manufacturers: ['Aftab Automobiles', 'PHP Automobiles', 'Aftab Automobiles', 'PHP Automobiles']

Country: Belarus[edit]
Car Manufacturers: ['BelAZ', 'BelGee', 'MAZ', 'MoAZ', 'MZKT', 'Neman', 'BelAZ', 'BelGee', 'MAZ', 'MoAZ', 'MZKT', 'Neman']

Coun